# Kia Georgia Interview (Chanyoung Park): Analyze Claim Data

## Dataset Information
I downloaded the customer complaint data from NHTSA (National Highway Traffic Safety Administration). 

Complaint information entered into NHTSA’s Office of Defects Investigation vehicle owner's complaint database is used with other data sources to identify safety issues that warrant investigation and to determine if a safety-related defect trend exists. Complaint information is also analyzed to monitor existing recalls for proper scope and adequacy.

---

## Purpose of the Analysis
The primary goal of this analysis is to **identify patterns and trends in warranty claims and vehicle defects reported by customers**. While the dataset does not include explicit supplier (supplier-related) information, it provides valuable insights into defects associated with specific vehicle models.

For this analysis, I focus on **Kia Georgia, Inc.'s production models**: **Telluride, Sorento, and Sportage**, which are manufactured in the West Point, GA facility. 

By examining the defects and complaints related to these models, I aim to:
- Understand the most frequently occurring and high-cost defects for these vehicles.
- Provide insights into potential supplier-related issues, even though direct supplier data is not available.
- Support Kia Georgia's efforts to **reclaim warranty expenses from suppliers** and improve production quality by identifying recurring defect patterns.

This analysis aligns closely with the responsibilities of the Bilingual Quality Assurance Specialist (Reimbursement) role, as it provides actionable insights to assist in **negotiating supplier burden ratios** and **tracking recall costs**.


### Data Load

In [1]:
import pandas as pd

# Load the CSV file with encoding
df = pd.read_csv('C:/Users/cytim/OneDrive/바탕 화면/미국 취업 준비 자료/quality_claim1.csv', encoding='ISO-8859-1')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\1856989783.py:4: DtypeWarning: Columns (22,26,27,28,29,30,31,33,38,41,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/cytim/OneDrive/바탕 화면/미국 취업 준비 자료/quality_claim1.csv', encoding='ISO-8859-1')


In [2]:
df.head()

,1633296,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018,N,20191221,N.1,0,...,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,V,Unnamed: 46,N.6,N.7
0,1633297,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018.0,N,20191221,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
1,1633298,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018.0,N,20191221,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
2,1633299,11292385,Ford Motor Company,FORD,EXPLORER,2020.0,N,20191226,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
3,1633300,11292386,"General Motors, LLC",CHEVROLET,VOLT,2017.0,N,20190712,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
4,1633301,11292386,"General Motors, LLC",CHEVROLET,VOLT,2017.0,N,20190712,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N


In [3]:
df.shape

(404722, 49)

In [4]:
df.info

<bound method DataFrame.info of         1633296  11292384  Honda (American Honda Motor Co.)      HONDA  \
0       1633297  11292384  Honda (American Honda Motor Co.)      HONDA   
1       1633298  11292384  Honda (American Honda Motor Co.)      HONDA   
2       1633299  11292385                Ford Motor Company       FORD   
3       1633300  11292386               General Motors, LLC  CHEVROLET   
4       1633301  11292386               General Motors, LLC  CHEVROLET   
...         ...       ...                               ...        ...   
404717  2038014  11624120        Nissan North America, Inc.     NISSAN   
404718  2038015  11624121                       Tesla, Inc.      TESLA   
404719  2038016  11624121                       Tesla, Inc.      TESLA   
404720  2038017  11624121                       Tesla, Inc.      TESLA   
404721  2038018  11624122          Toyota Motor Corporation     TOYOTA   

          ACCORD    2018  N  20191221 N.1  0  ...  Unnamed: 39 Unnamed: 40  \
0

### Data preprocessing

In [5]:
# Drop columns that are either completely empty or have irrelevant placeholder names
columns_to_drop = [col for col in df.columns if "Unnamed" in col or col.startswith("N")]
df.drop(columns=columns_to_drop, inplace=True)

# Rename the columns based on their content
df.columns = [
    "ComplaintID", "VehicleID", "Manufacturer", "Make", "Model", "ModelYear",
    "DateFiled", "Unknown1", "Unknown2", "DefectType", "City", "State",
    "VIN", "RepairDate", "InspectionDate", "Mileage", "ComplaintDescription",
    "Source", "SeverityScore", "Status"
]

# Display the renamed columns and the first few rows to verify
print(df.columns)
print(df.head())

Index(['ComplaintID', 'VehicleID', 'Manufacturer', 'Make', 'Model',
       'ModelYear', 'DateFiled', 'Unknown1', 'Unknown2', 'DefectType', 'City',
       'State', 'VIN', 'RepairDate', 'InspectionDate', 'Mileage',
       'ComplaintDescription', 'Source', 'SeverityScore', 'Status'],
      dtype='object')
   ComplaintID  VehicleID                      Manufacturer       Make  \
0      1633297   11292384  Honda (American Honda Motor Co.)      HONDA   
1      1633298   11292384  Honda (American Honda Motor Co.)      HONDA   
2      1633299   11292385                Ford Motor Company       FORD   
3      1633300   11292386               General Motors, LLC  CHEVROLET   
4      1633301   11292386               General Motors, LLC  CHEVROLET   

      Model  ModelYear  DateFiled  Unknown1  Unknown2  \
0    ACCORD     2018.0   20191221         0         0   
1    ACCORD     2018.0   20191221         0         0   
2  EXPLORER     2020.0   20191226         0         0   
3      VOLT     2017.0 

In [6]:
df['DateFiled'] = pd.to_datetime(df['DateFiled'], format='%Y%m%d', errors='coerce')
df['RepairDate'] = pd.to_datetime(df['RepairDate'], format='%Y%m%d', errors='coerce')
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'], format='%Y%m%d', errors='coerce')

In [7]:
df.drop(columns=['Unknown1', 'Unknown2'], inplace=True)

In [8]:
df.head()

,ComplaintID,VehicleID,Manufacturer,Make,Model,ModelYear,DateFiled,DefectType,City,State,VIN,RepairDate,InspectionDate,Mileage,ComplaintDescription,Source,SeverityScore,Status
0,1633297,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018.0,2019-12-21,ELECTRICAL SYSTEM,PHILADELPHIA,PA,1HGCV2F38JA,2020-01-01,2020-01-01,4.0,"DRIVING AT THE HIGHWAY, CAR SUDDENLY SLOW DOWN...",IVOQ,68.0,V
1,1633298,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018.0,2019-12-21,ENGINE,PHILADELPHIA,PA,1HGCV2F38JA,2020-01-01,2020-01-01,4.0,"DRIVING AT THE HIGHWAY, CAR SUDDENLY SLOW DOWN...",IVOQ,68.0,V
2,1633299,11292385,Ford Motor Company,FORD,EXPLORER,2020.0,2019-12-26,ELECTRICAL SYSTEM,MEHERRIN,VA,1FM5K8GC8LG,2020-01-01,2020-01-01,5300.0,DEEP SLEEP MODE ACTIVATES AFTER 2 DAYS. MOST ...,IVOQ,0.0,V
3,1633300,11292386,"General Motors, LLC",CHEVROLET,VOLT,2017.0,2019-07-12,SERVICE BRAKES,SAN ANTONIO,TX,1G1RB6S52HU,2020-01-01,2020-01-01,15000.0,"WHILE DRIVING ON CITY STREETS AND HIGHWAYS, TH...",IVOQ,70.0,V
4,1633301,11292386,"General Motors, LLC",CHEVROLET,VOLT,2017.0,2019-07-12,ELECTRONIC STABILITY CONTROL,SAN ANTONIO,TX,1G1RB6S52HU,2020-01-01,2020-01-01,15000.0,"WHILE DRIVING ON CITY STREETS AND HIGHWAYS, TH...",IVOQ,70.0,V


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404722 entries, 0 to 404721
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ComplaintID           404722 non-null  int64         
 1   VehicleID             404722 non-null  int64         
 2   Manufacturer          404715 non-null  object        
 3   Make                  404715 non-null  object        
 4   Model                 404715 non-null  object        
 5   ModelYear             404715 non-null  float64       
 6   DateFiled             404722 non-null  datetime64[ns]
 7   DefectType            404720 non-null  object        
 8   City                  404706 non-null  object        
 9   State                 404722 non-null  object        
 10  VIN                   395787 non-null  object        
 11  RepairDate            404722 non-null  datetime64[ns]
 12  InspectionDate        404722 non-null  datetime64[ns]
 13 

In [10]:
df.isnull().sum()

ComplaintID                  0
VehicleID                    0
Manufacturer                 7
Make                         7
Model                        7
ModelYear                    7
DateFiled                    0
DefectType                   2
City                        16
State                        0
VIN                       8935
RepairDate                   0
InspectionDate               0
Mileage                 257128
ComplaintDescription        17
Source                       7
SeverityScore           172421
Status                     356
dtype: int64

In [11]:
import pandas as pd

# Step 1: Replace abnormal values in 'ModelYear' with NaN
df['ModelYear'] = df['ModelYear'].replace(9999, pd.NA)

# Step 2: Filter rows for specific date range in 'DateFiled'
df = df[(df['DateFiled'] >= '2020-01-01') & (df['DateFiled'] <= '2024-12-31')]

# Step 3: Replace abnormal values in 'Mileage' with NaN
df['Mileage'] = df['Mileage'].replace([-1, 0], pd.NA)

# Step 4: Remove leading/trailing whitespace in the 'City' column
df['City'] = df['City'].str.strip()

# Step 5: Focus on recent vehicle models by filtering for 'ModelYear' >= 2015
df = df[df['ModelYear'] >= 2015]

# Step 6: Drop rows with essential information missing
df.dropna(subset=['Manufacturer', 'Make', 'Model', 'ModelYear'], inplace=True)

# Step 7: Fill missing values with placeholders for categorical columns
df['DefectType'].fillna('UNKNOWN', inplace=True)
df['City'].fillna('UNKNOWN', inplace=True)
df['ComplaintDescription'].fillna('No description provided', inplace=True)
df['Source'].fillna('UNKNOWN', inplace=True)
df['Status'].fillna('UNKNOWN', inplace=True)

# Step 8: Fill missing VIN with a placeholder
df['VIN'].fillna('UNKNOWN', inplace=True)

# Step 9: Convert SeverityScore to numeric type, forcing errors to NaN
df['SeverityScore'] = pd.to_numeric(df['SeverityScore'], errors='coerce')

# Step 10: Fill missing SeverityScore with median value
severity_median = df['SeverityScore'].median()
df['SeverityScore'].fillna(severity_median, inplace=True)

# Step 11: Fill missing Mileage with median value instead of placeholder
mileage_median = df['Mileage'].median()
df['Mileage'].fillna(mileage_median, inplace=True)

# Step 12: Remove rows where any column has 'UNKNOWN' as a value
df = df[
    (df['City'] != 'UNKNOWN') &
    (df['VIN'] != 'UNKNOWN') &
    (df['ComplaintDescription'] != 'No description provided') &
    (df['Source'] != 'UNKNOWN') &
    (df['Status'] != 'UNKNOWN')
]

# Step 13: Drop rows with remaining missing values (if critical columns still have NaN)
df.dropna(inplace=True)

# Final check for missing values
print("Remaining missing values after processing:")
print(df.isnull().sum())

# Final data size
print(f"Final number of rows: {len(df)}")


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\117328983.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Mileage'] = df['Mileage'].replace([-1, 0], pd.NA)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\117328983.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'] = df['City'].str.strip()


Remaining missing values after processing:
ComplaintID             0
VehicleID               0
Manufacturer            0
Make                    0
Model                   0
ModelYear               0
DateFiled               0
DefectType              0
City                    0
State                   0
VIN                     0
RepairDate              0
InspectionDate          0
Mileage                 0
ComplaintDescription    0
Source                  0
SeverityScore           0
Status                  0
dtype: int64
Final number of rows: 252838


In [12]:
df.isnull().sum()

ComplaintID             0
VehicleID               0
Manufacturer            0
Make                    0
Model                   0
ModelYear               0
DateFiled               0
DefectType              0
City                    0
State                   0
VIN                     0
RepairDate              0
InspectionDate          0
Mileage                 0
ComplaintDescription    0
Source                  0
SeverityScore           0
Status                  0
dtype: int64

In [13]:
#df.to_csv('C:/Users/cytim/OneDrive/바탕 화면/미국 취업 준비 자료/cleaned_quality_claim.csv', index=False)

In [14]:
df1 = pd.read_csv('C:/Users/cytim/OneDrive/바탕 화면/미국 취업 준비 자료/cleaned_quality_claim.csv', encoding='ISO-8859-1')

In [15]:
df1.head(5)

,ComplaintID,VehicleID,Manufacturer,Make,Model,ModelYear,DateFiled,DefectType,City,State,VIN,RepairDate,InspectionDate,Mileage,ComplaintDescription,Source,SeverityScore,Status
0,1633305,11292389,Hyundai Motor America,HYUNDAI,AZERA,2015.0,2020-01-01,SERVICE BRAKES,PELHAM,AL,KMHFH4JG0FA,2020-01-01,2020-01-01,80000.0,HIT A BUMP IN ROAD AND LOST CONTROL OF CAR. UN...,IVOQ,65.0,V
1,1633306,11292389,Hyundai Motor America,HYUNDAI,AZERA,2015.0,2020-01-01,STEERING,PELHAM,AL,KMHFH4JG0FA,2020-01-01,2020-01-01,80000.0,HIT A BUMP IN ROAD AND LOST CONTROL OF CAR. UN...,IVOQ,65.0,V
2,1633307,11292389,Hyundai Motor America,HYUNDAI,AZERA,2015.0,2020-01-01,SUSPENSION,PELHAM,AL,KMHFH4JG0FA,2020-01-01,2020-01-01,80000.0,HIT A BUMP IN ROAD AND LOST CONTROL OF CAR. UN...,IVOQ,65.0,V
3,1633308,11292390,"Kia America, Inc.",KIA,SOUL,2020.0,2020-01-01,ENGINE,KANKAKEE,IL,KNDJ23AU0L7,2020-01-01,2020-01-01,45000.0,THE CAR WOULD SHUT OFF AND I WILL START IT BAC...,IVOQ,35.0,V
4,1633309,11292390,"Kia America, Inc.",KIA,SOUL,2020.0,2020-01-01,UNKNOWN OR OTHER,KANKAKEE,IL,KNDJ23AU0L7,2020-01-01,2020-01-01,45000.0,THE CAR WOULD SHUT OFF AND I WILL START IT BAC...,IVOQ,35.0,V


In [16]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252838 entries, 0 to 252837
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ComplaintID           252838 non-null  int64  
 1   VehicleID             252838 non-null  int64  
 2   Manufacturer          252838 non-null  object 
 3   Make                  252838 non-null  object 
 4   Model                 252838 non-null  object 
 5   ModelYear             252838 non-null  float64
 6   DateFiled             252838 non-null  object 
 7   DefectType            252838 non-null  object 
 8   City                  252838 non-null  object 
 9   State                 252838 non-null  object 
 10  VIN                   252838 non-null  object 
 11  RepairDate            252838 non-null  object 
 12  InspectionDate        252838 non-null  object 
 13  Mileage               252838 non-null  float64
 14  ComplaintDescription  252838 non-null  object 
 15  

## Step 1: Analyzing Claims for All Kia Vehicles

### Objective
The first step in this analysis focuses on understanding warranty claims and defect trends across **all Kia vehicles**. By analyzing claims at this broader level, we can identify:
- The most common defect types reported for Kia vehicles.
- Patterns in claims based on factors such as **vehicle models**, **defect types**, and **locations**.
- High-severity defects that contribute significantly to warranty costs.

In [17]:
# Kia data filtering
kia_df = df1[df1['Manufacturer'] == 'Kia America, Inc.']
print(f"Kia data size: {kia_df.shape}")

Kia data size: (13976, 18)


### Analysis Process
**Defect Type Frequency Analysis**:
   - Identify the most frequently reported defect types for Kia vehicles.
   - Highlight high-cost defect types based on SeverityScore.



In [18]:
# Defect Type Frequency
defect_type_frequency = kia_df['DefectType'].value_counts()

# High-severity Defects
high_severity_defects = kia_df[kia_df['SeverityScore'] > kia_df['SeverityScore'].median()]
high_severity_defect_types = high_severity_defects['DefectType'].value_counts()

print("Defect Type Frequency:")
print(defect_type_frequency.head(10))

print("\nHigh-Severity Defect Types:")
print(high_severity_defect_types.head(10))

Defect Type Frequency:
ENGINE                    4461
UNKNOWN OR OTHER          1730
ELECTRICAL SYSTEM         1420
POWER TRAIN                951
EXTERIOR LIGHTING          584
FUEL/PROPULSION SYSTEM     520
STRUCTURE:BODY             474
SERVICE BRAKES             414
STEERING                   386
VISIBILITY/WIPER           381
Name: DefectType, dtype: int64

High-Severity Defect Types:
ENGINE                    1640
UNKNOWN OR OTHER           437
POWER TRAIN                320
ELECTRICAL SYSTEM          313
EXTERIOR LIGHTING          168
STEERING                   156
FUEL/PROPULSION SYSTEM     149
VEHICLE SPEED CONTROL      137
VISIBILITY/WIPER           132
SERVICE BRAKES             114
Name: DefectType, dtype: int64


**Model-Specific Defect Analysis**:
   - Analyze claims for Kia's various vehicle models to understand model-specific defect patterns.
   - Focus on recurring defect types associated with certain models.

In [19]:
# Group by Make, Model, and ModelYear to count defects
model_defects = kia_df.groupby(['Make', 'Model', 'ModelYear'])['ComplaintID'].count()

# Sort by highest defect frequency
model_defects_sorted = model_defects.sort_values(ascending=False)

print("Top Models with the Most Defects:")
print(model_defects_sorted.head(10))

Top Models with the Most Defects:
Make  Model      ModelYear
KIA   SORENTO    2016.0       1031
      OPTIMA     2015.0        801
      SOUL       2016.0        746
                 2015.0        719
      OPTIMA     2016.0        677
      SORENTO    2017.0        609
                 2015.0        536
      TELLURIDE  2020.0        500
      SORENTO    2019.0        342
      SPORTAGE   2017.0        293
Name: ComplaintID, dtype: int64


**Location-Based Defect Analysis**:
   - Investigate claim data by **state** and **city** to determine regional trends.
   - Identify regions with higher frequencies of high-severity claims.

In [20]:
# State-Level Defect Frequency
state_defects = kia_df['State'].value_counts()

# City-Level Defect Frequency
city_defects = kia_df.groupby(['City', 'State'])['ComplaintID'].count().sort_values(ascending=False)

print("State-Level Defect Frequency:")
print(state_defects.head(10))

print("\nCity-Level Defect Frequency:")
print(city_defects.head(10))

State-Level Defect Frequency:
CA    1510
FL    1335
TX    1154
GA     808
OH     616
NC     573
PA     554
IL     531
NY     525
VA     456
Name: State, dtype: int64

City-Level Defect Frequency:
City          State
Monroe        MI       114
Chicago       IL        89
Orlando       FL        65
Houston       TX        62
Los Angeles   CA        55
Jacksonville  FL        48
San Antonio   TX        43
Dallas        TX        41
Cincinnati    OH        40
San Diego     CA        40
Name: ComplaintID, dtype: int64


**Repair Time Analysis**:
   - Calculate average repair durations for different defect types.
   - Highlight defects associated with longer repair times, which could indicate supplier-related challenges.

In [21]:
# Calculate Repair Duration
kia_df['RepairDate'] = pd.to_datetime(kia_df['RepairDate'])
kia_df['DateFiled'] = pd.to_datetime(kia_df['DateFiled'])
kia_df['RepairDuration'] = (kia_df['RepairDate'] - kia_df['DateFiled']).dt.days

# Average Repair Duration by Defect Type
repair_duration_by_defect = kia_df.groupby('DefectType')['RepairDuration'].mean().sort_values(ascending=False)

print("Average Repair Duration by Defect Type:")
print(repair_duration_by_defect.head(10))

Average Repair Duration by Defect Type:
DefectType
PARKING BRAKE:ELECTRICAL                                     924.0
VISIBILITY:REARVIEW MIRRORS/DEVICES:EXTERIOR                 721.5
STEERING:STEERING WHEEL /HANDLE BAR:YAW/ANGLE SENSOR         666.0
SEAT BELTS: REAR/OTHER:BUCKLE ASSEMBLY                       639.0
FORWARD COLLISION AVOIDANCE: SENSING SYSTEM: RADAR           421.0
EXTERIOR LIGHTING:BRAKE LIGHTS:BULBS                         397.0
FUEL SYSTEM, GASOLINE:DELIVERY:FUEL PUMP                     396.5
EXTERIOR LIGHTING:TAIL LIGHTS:BULBS                          396.0
EXTERIOR LIGHTING:HEADLIGHTS:DAYTIME RUNNING LIGHTS:BULBS    396.0
FUEL SYSTEM, GASOLINE:FUEL INJECTION SYSTEM                  341.0
Name: RepairDuration, dtype: float64


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\977823697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_df['RepairDate'] = pd.to_datetime(kia_df['RepairDate'])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\977823697.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_df['DateFiled'] = pd.to_datetime(kia_df['DateFiled'])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\977823697.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

**Mileage and Cost Analysis**:
   - Examine claims based on vehicle mileage to identify defect trends across different mileage ranges.
   - Assess the average cost associated with claims at various mileage levels.

In [22]:
# Group by Mileage Range and calculate average SeverityScore
mileage_bins = [0, 5000, 10000, 20000, 50000, 100000]
mileage_labels = ['0-5k', '5k-10k', '10k-20k', '20k-50k', '50k-100k']
kia_df['MileageRange'] = pd.cut(kia_df['Mileage'], bins=mileage_bins, labels=mileage_labels)

mileage_cost_analysis = kia_df.groupby('MileageRange')['SeverityScore'].mean()

print("Mileage and Cost Analysis:")
print(mileage_cost_analysis)

Mileage and Cost Analysis:
MileageRange
0-5k        39.485934
5k-10k      33.926667
10k-20k     36.399225
20k-50k     36.439369
50k-100k    36.760719
Name: SeverityScore, dtype: float64


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\1470790845.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_df['MileageRange'] = pd.cut(kia_df['Mileage'], bins=mileage_bins, labels=mileage_labels)


**Time-Based Trend Analysis**:
   - Analyze claim trends over time to understand if defects are increasing or decreasing for Kia vehicles.
   - Identify any potential correlations between time and high-severity defects.

In [23]:
# Extract year from DateFiled
kia_df['year'] = kia_df['DateFiled'].dt.year

# Group by year to calculate average SeverityScore
time_trend_analysis = kia_df.groupby('year')['SeverityScore'].mean()

print("Time-Based Trend Analysis:")
print(time_trend_analysis)

Time-Based Trend Analysis:
year
2020    35.740043
2021    37.667254
2022    37.032177
2023    36.830460
2024    35.955674
Name: SeverityScore, dtype: float64


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\2889234742.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_df['year'] = kia_df['DateFiled'].dt.year


## Step 2: Analyzing Claims for Kia Georgia Production Models

### Objective
This section focuses on analyzing warranty claims and defect trends for **Kia Georgia's production models**: **Sorento**, **Telluride**, and **Sportage**. These models are exclusively manufactured at the Kia Georgia facility in West Point, GA.

The analysis aims to:
- Identify **defect types** and **their frequencies** specific to these models.
- Highlight **high-severity defects** that may significantly impact warranty costs.
- Analyze **regional trends** and **repair durations** for these models.
- Provide insights into defects based on **mileage ranges** and **time trends**.

### Analysis Process
To meet these objectives, the analysis is broken down into the following steps:

**Defect Type Frequency Analysis**:
   - Identify the most common defect types for Sorento, Telluride, and Sportage.
   - Focus on high-severity defect types.

In [24]:
# Filter data for Sorento, Telluride, and Sportage
kia_georgia_models = df1[df1['Model'].isin(['SORENTO', 'TELLURIDE', 'SPORTAGE'])]

# Defect Type Frequency
defect_type_frequency = kia_georgia_models['DefectType'].value_counts()

# High-severity Defects
high_severity_defects = kia_georgia_models[kia_georgia_models['SeverityScore'] > kia_georgia_models['SeverityScore'].median()]
high_severity_defect_types = high_severity_defects['DefectType'].value_counts()

print("Defect Type Frequency for Sorento, Telluride, and Sportage:")
print(defect_type_frequency.head(10))

print("\nHigh-Severity Defect Types for Sorento, Telluride, and Sportage:")
print(high_severity_defect_types.head(10))

Defect Type Frequency for Sorento, Telluride, and Sportage:
ENGINE                   1551
UNKNOWN OR OTHER          788
ELECTRICAL SYSTEM         528
POWER TRAIN               390
EXTERIOR LIGHTING         370
VISIBILITY/WIPER          301
STRUCTURE:BODY            271
STEERING                  187
SERVICE BRAKES            151
VEHICLE SPEED CONTROL     150
Name: DefectType, dtype: int64

High-Severity Defect Types for Sorento, Telluride, and Sportage:
ENGINE                   614
UNKNOWN OR OTHER         207
ELECTRICAL SYSTEM        125
POWER TRAIN              116
VISIBILITY/WIPER         109
EXTERIOR LIGHTING        108
STEERING                  94
VEHICLE SPEED CONTROL     61
STRUCTURE:BODY            45
SERVICE BRAKES            42
Name: DefectType, dtype: int64


**Location-Based Analysis**:
   - Determine which **states** and **cities** report the most claims for these models.

In [25]:
# State-Level Defect Frequency
state_defects = kia_georgia_models['State'].value_counts()

# City-Level Defect Frequency
city_defects = kia_georgia_models.groupby(['City', 'State'])['ComplaintID'].count().sort_values(ascending=False)

print("State-Level Defect Frequency for Sorento, Telluride, and Sportage:")
print(state_defects.head(10))

print("\nCity-Level Defect Frequency for Sorento, Telluride, and Sportage:")
print(city_defects.head(10))

State-Level Defect Frequency for Sorento, Telluride, and Sportage:
CA    524
FL    492
TX    477
GA    335
PA    284
NY    259
NC    246
OH    240
MI    229
IL    209
Name: State, dtype: int64

City-Level Defect Frequency for Sorento, Telluride, and Sportage:
City          State
Monroe        MI       114
Chicago       IL        38
Jacksonville  FL        24
Orlando       FL        21
Los Angeles   CA        21
Philadelphia  PA        21
Houston       TX        18
Warrenton     GA        16
Fort Worth    TX        16
Dallas        TX        16
Name: ComplaintID, dtype: int64


**Repair Time Analysis**:
   - Calculate the average repair duration for each defect type for Sorento, Telluride, and Sportage.

In [27]:
# Ensure 'RepairDate' and 'DateFiled' are in datetime format
kia_georgia_models['RepairDate'] = pd.to_datetime(kia_georgia_models['RepairDate'], errors='coerce')
kia_georgia_models['DateFiled'] = pd.to_datetime(kia_georgia_models['DateFiled'], errors='coerce')

# Calculate Repair Duration
kia_georgia_models['RepairDuration'] = (kia_georgia_models['RepairDate'] - kia_georgia_models['DateFiled']).dt.days

# Check for invalid or missing durations
if kia_georgia_models['RepairDuration'].isnull().sum() > 0:
    print("Warning: Some rows have missing or invalid RepairDuration values.")
    # Optionally, drop rows with invalid RepairDuration
    kia_georgia_models = kia_georgia_models.dropna(subset=['RepairDuration'])

# Average Repair Duration by Defect Type
repair_duration_by_defect = kia_georgia_models.groupby('DefectType')['RepairDuration'].mean().sort_values(ascending=False)

print("Average Repair Duration by Defect Type for Sorento, Telluride, and Sportage:")
print(repair_duration_by_defect.head(10))

# Average Repair Duration by Defect Type (shortest duration first)
repair_duration_by_defect_shortest = kia_georgia_models.groupby('DefectType')['RepairDuration'].mean().sort_values(ascending=True)

print("Defect Types with the Shortest Average Repair Duration for Sorento, Telluride, and Sportage:")
print(repair_duration_by_defect_shortest.head(10))


Average Repair Duration by Defect Type for Sorento, Telluride, and Sportage:
DefectType
PARKING BRAKE:ELECTRICAL                                                         924.000000
EXTERIOR LIGHTING:BRAKE LIGHTS:BULBS                                             791.000000
VISIBILITY:REARVIEW MIRRORS/DEVICES:EXTERIOR                                     721.500000
SEAT BELTS: REAR/OTHER:BUCKLE ASSEMBLY                                           454.000000
FORWARD COLLISION AVOIDANCE: SENSING SYSTEM: RADAR                               421.000000
ENGINE AND ENGINE COOLING:EXHAUST SYSTEM:EMISSION CONTROL:CATALYTIC CONVERTOR    396.333333
FUEL SYSTEM, GASOLINE:DELIVERY:FUEL PUMP                                         287.000000
ENGINE AND ENGINE COOLING:ENGINE:OIL/LUBRICATION                                 280.600000
CHILD SEAT:VEHICLE LATCH ANCHOR                                                  270.000000
LANE DEPARTURE: SENSING SYSTEM: SONAR/ULTRASONIC                                 254

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\3883458698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_georgia_models['RepairDate'] = pd.to_datetime(kia_georgia_models['RepairDate'], errors='coerce')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\3883458698.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_georgia_models['DateFiled'] = pd.to_datetime(kia_georgia_models['DateFiled'], errors='coerce')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\

In [28]:
kia_georgia_models['RepairDurationRange'] = pd.cut(
    kia_georgia_models['RepairDuration'],
    bins=[0, 30, 90, 180, 365],
    labels=['0-30 days', '30-90 days', '90-180 days', '180-365 days']
)
print(kia_georgia_models['RepairDurationRange'].value_counts())

0-30 days       2864
30-90 days       864
90-180 days      494
180-365 days     368
Name: RepairDurationRange, dtype: int64


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\2868765286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_georgia_models['RepairDurationRange'] = pd.cut(


**Mileage-Based Analysis**:
   - Analyze how defect types vary across different mileage ranges for these models.
   - Highlight high-cost defects within each mileage range.

In [29]:
# Group by Mileage Range and calculate average SeverityScore
mileage_bins = [0, 5000, 10000, 20000, 50000, 100000]
mileage_labels = ['0-5k', '5k-10k', '10k-20k', '20k-50k', '50k-100k']
kia_georgia_models['MileageRange'] = pd.cut(kia_georgia_models['Mileage'], bins=mileage_bins, labels=mileage_labels)

mileage_cost_analysis = kia_georgia_models.groupby('MileageRange')['SeverityScore'].mean()

print("Mileage and Cost Analysis for Sorento, Telluride, and Sportage:")
print(mileage_cost_analysis)

Mileage and Cost Analysis for Sorento, Telluride, and Sportage:
MileageRange
0-5k        47.107383
5k-10k      34.347826
10k-20k     37.495327
20k-50k     36.949666
50k-100k    36.774834
Name: SeverityScore, dtype: float64


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\1048713566.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_georgia_models['MileageRange'] = pd.cut(kia_georgia_models['Mileage'], bins=mileage_bins, labels=mileage_labels)


**Time-Based Trend Analysis**:
   - Examine how claims and defects for these models have evolved over the years.

In [30]:
# Extract year from DateFiled
kia_georgia_models['year'] = kia_georgia_models['DateFiled'].dt.year

# Group by year to calculate average SeverityScore
kia_model_yearly_defects = kia_georgia_models.groupby(['Model', 'year']).size()
time_trend_analysis = kia_georgia_models.groupby(['Model','year'])['SeverityScore'].mean()

print("Yearly defects for Sorento, Telluride, and Sportage:")
print(kia_model_yearly_defects)

print("Severity Score for Sorento, Telluride, and Sportage:")
print(time_trend_analysis)

Yearly defects for Sorento, Telluride, and Sportage:
Model      year
SORENTO    2020    449
           2021    544
           2022    876
           2023    885
           2024    572
SPORTAGE   2020    102
           2021    147
           2022    331
           2023    349
           2024    211
TELLURIDE  2020    100
           2021    174
           2022    243
           2023    423
           2024    396
dtype: int64
Severity Score for Sorento, Telluride, and Sportage:
Model      year
SORENTO    2020    38.144766
           2021    38.314338
           2022    37.519406
           2023    38.447458
           2024    39.389860
SPORTAGE   2020    35.500000
           2021    44.326531
           2022    34.141994
           2023    39.220630
           2024    40.052133
TELLURIDE  2020    38.840000
           2021    37.649425
           2022    37.572016
           2023    33.900709
           2024    30.739899
Name: SeverityScore, dtype: float64


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35544\1555536568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kia_georgia_models['year'] = kia_georgia_models['DateFiled'].dt.year


## Insights from Step 2 Analysis: Sorento, Telluride, and Sportage Defect Analysis

### 1. Defect Type Frequency
- The **ENGINE** defect is the most common among Sorento, Telluride, and Sportage, with 1,551 occurrences.
- **UNKNOWN OR OTHER** defects (788) and **ELECTRICAL SYSTEM** defects (528) also rank high, highlighting potential areas for further investigation.
- **Powertrain** (390), **Exterior Lighting** (370), and **Visibility/Wiper** (301) are additional frequent defect types.
- **Insight**: **Engine-related defects** should be prioritized for supplier negotiations and quality improvement, as they dominate the defect categories.

---

### 2. High-Severity Defects
- **ENGINE** defects also top the list for high-severity issues with 614 cases, indicating significant cost implications.
- Other high-severity defect types include **UNKNOWN OR OTHER** (207), **ELECTRICAL SYSTEM** (125), and **POWER TRAIN** (116).
- **Insight**: High-severity defects should be the focus for **cost-sharing negotiations with suppliers**, especially for **Engine**, **Electrical**, and **Powertrain** issues.

---

### 3. State-Level Defect Frequency
- **California (CA), Florida (FL), and Texas (TX)** report the highest defect counts, with **CA leading with 524 cases**.
- Georgia (GA), where Kia Georgia is located, ranks 4th with **335 defects**, demonstrating the importance of focusing on local recall and claim management.
- **Insight**: Specific strategies for **high-defect states** (CA, FL, TX, and GA) can optimize resource allocation for recalls and defect resolution.

---

### 4. City-Level Defect Frequency
- **Monroe, MI**, is a hotspot for defect complaints, with **114 cases**, followed by Chicago, IL (38), and Jacksonville, FL (24).
- **Insight**: Cities with high defect counts can be targeted for **localized campaigns or service enhancements**.

---

### 5. Average Repair Duration by Defect Type
- Some defect types, such as:
  - **Parking Brake: Electrical (924 days)** and 
  - **Exterior Lighting: Brake Lights (791 days)**, show **unusually long repair durations**.
- Other notable defect types with high durations include:
  - **Visibility: Rearview Mirrors (721.5 days)** and 
  - **Seat Belts: Buckle Assembly (454 days)**.
- **Insight**: Long repair durations may indicate **inefficiencies in parts supply or repair processes**. These should be flagged for process improvement or supplier accountability.

#### Repair Duration part I have a doubt, so keep thinking about is this statistic is real or not. 
---

### 6. Mileage and Cost Analysis
- Severity scores vary by mileage range, with:
  - **0-5k miles**: Highest severity score (**47.11**).
  - **5k-10k miles**: Drop in severity (**34.35**).
  - **10k-20k miles and beyond**: Scores stabilize between **36-37**.
- **Insight**: Defects in **newer vehicles (low mileage)** tend to be more severe, warranting further investigation of manufacturing defects or early-life failures.

---

### 7. Yearly Defects by Model
- **Sorento** consistently has the highest defect counts, peaking in 2023 (**885 cases**).
- **Telluride's** defect count has risen steadily from 2020 (100) to 2024 (396).
- **Sportage** defects peaked in 2023 (349 cases) but remain lower overall compared to Sorento and Telluride.
- **Insight**: **Sorento and Telluride are key models** to monitor for quality improvements, with consistent defect counts across multiple years.

---

### 8. Severity Score Trends by Year and Model
- **Sorento**:
  - Severity scores are increasing, peaking at **39.39** in 2024, suggesting worsening defects over time.
- **Telluride**:
  - Severity scores show a declining trend, dropping to **30.74** in 2024, indicating some improvements.
- **Sportage**:
  - Severity scores fluctuate, peaking at **40.05** in 2024.
- **Insight**: The **increasing severity trend for Sorento and Sportage** highlights areas needing urgent attention, while Telluride shows progress in defect severity reduction.

---

## Actionable Recommendations

1. **Prioritize High-Severity Defects**:
   - Focus on **Engine**, **Electrical System**, and **Powertrain** issues, especially for Sorento, as they dominate both frequency and severity.

2. **Target High-Defect Regions**:
   - Devise localized strategies for **California, Florida, Texas, and Georgia**, which report the highest defect counts.

3. **Investigate Long Repair Durations**:
   - Address inefficiencies for defects with prolonged repair durations, particularly for **Parking Brake**, **Exterior Lighting**, and **Visibility** issues.

4. **Examine Early-Life Failures**:
   - Defects in vehicles with lower mileage (0-5k miles) have the highest severity, suggesting a need to analyze **early manufacturing defects**.

5. **Monitor Yearly Trends**:
   - Sorento’s increasing severity scores need immediate intervention.
   - Leverage Telluride's declining severity trend as a model for improving other vehicles.

6. **Collaborate with Suppliers**:
   - Use the insights to engage **suppliers** in **cost-sharing negotiations** for high-severity and high-frequency defects, focusing on accountability for engine-related issues.

---